In [2]:
import glob
import zipfile 
import os
import xml.etree.ElementTree as ET
import collections
import csv


In [3]:
def read_decisions(root, path, pairs):
    pairs_renseignement = []
    pairs_titulaire = []
    for element in root.findall(path):
        for child in element:
            
            if child.text is not None: 
                pairs.append([child.tag, child.text])
                
            if child.tag == 'RENSEIGNEMENT':
                for hijo in child:
                    pairs_renseignement.append([hijo.tag, hijo.text])
                    
            if child.tag == 'TITULAIRE':
                for hijo in child:
                    pairs_titulaire.append([hijo.tag, hijo.text])
                    
    return pairs, pairs_renseignement, pairs_titulaire

In [4]:
def extract_nicaa(file):
    
    pairs = []
    tree = ET.parse(file)
    root = tree.getroot()
    
    paths_nicaa = ['GESTION/REFERENCE/IDWEB',
                   'GESTION/MARCHE/ANNONCE_ANTERIEUR/REFERENCE/IDWEB',
                   'DONNEES/IDENTITE/',
                   'DONNEES/OBJET/OBJET_COMPLET']
    
    for path in paths_nicaa:
        for element in root.findall(path):
            pairs.append([element.tag, element.text])
            
    pairs_renseignement, pairs_titulaire, pairs = read_decisions(root, 'DONNEES/ATTRIBUTION/', pairs)
    return pairs, pairs_renseignement, pairs_titulaire


In [62]:
def clean_dic(dic):
    new_dic = {}
    for key in dic.keys():
        new_container = []
        for pair in dic[key]:
            new_pair = []
            for element in pair:
                if element is not None:
                    new_pair.append(element.replace(',', ''))
            new_container.append(new_pair)
        new_dic[key] = new_container
    return new_dic

In [117]:
def write_nicaa_to_csv(nicaa, filename, n):
    nicaa = clean_dic(nicaa)
    if not os.path.exists('extraction_tests2K5/{}'.format(n)):
            os.makedirs('extraction_tests2K5/{}'.format(n))
    with open('extraction_tests2K5/{}/{}.csv'.format(n, filename), 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                                quotechar=' ', quoting=csv.QUOTE_ALL)
        for key in nicaa.keys():
            for pair in nicaa[key]:
                spamwriter.writerow(pair)

In [97]:
def compile_nicaa_folder(folder, n):
    lp = []
    files = glob.glob('{}/*.xml'.format(folder))
    for f in files:
        p, p1, p2 = extract_nicaa(f) 
        lp.append({'pairs': p, 'pairs_renseignement': p1, 'pairs_titulaire': p2 })
    for i in range(len(lp)):
        write_nicaa_to_csv(lp[i], i, n)
        

In [94]:
folders = glob.glob('/home/guillaume/Documents/BOAMP SHIT/2017/BOAMP-N-IC-AA*') 

In [109]:
files = glob.glob('/home/guillaume/Documents/BOAMP SHIT/test/xml/*')

In [111]:
files[0]

'/home/guillaume/Documents/BOAMP SHIT/test/xml/16-33134.xml'

In [118]:
compile_nicaa_folder('/home/guillaume/Documents/BOAMP SHIT/test/xml', 0)

In [103]:
all = glob.glob('extraction_tests2K17/*/*.csv')

In [104]:
len(all)

11744